In [ ]:
pip install opencv-python

Importing the necessary libraries..

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
from google.colab import drive
drive.mount('/content/drive')



  loading the image

In [ ]:
img = image.load_img('/content/drive/MyDrive/datasets/image classification/basedata/training/flood/image_101.jpg')

Showing the image

In [ ]:
plt.imshow(img)

To show the RGB values from 0 to 255

In [ ]:
cv2.imread('/content/drive/MyDrive/datasets/image classification/basedata/training/flood/image_101.jpg')

For shape

In [ ]:
cv2.imread('/content/drive/MyDrive/datasets/image classification/basedata/training/flood/image_101.jpg').shape

Generating training and validation dataset.

Resizing the values from 0 to 255 to 0s and 1s.

In [ ]:
train = ImageDataGenerator(rescale=1/255)
validation = ImageDataGenerator(rescale=1/255)

Giving labels to the images in the folder

Resizing the images to 200x200 to avoid conflicts 

batch_size defines how many images we train at one go

*   when it was 10 : prediction was very poor
*   After it was changed 90 : prediction was drastically improved 

class_mode defines how many labels we have.In this case we have only 2 so, its "binary"

For classification:

In [ ]:

# training
train_dataset=train.flow_from_directory('/content/drive/MyDrive/datasets/image classification/basedata/training/', target_size=(200,200), batch_size=90, class_mode='binary')

                                        
# validation
validation_dataset=train.flow_from_directory('/content/drive/MyDrive/datasets/image classification/basedata/validation/', target_size=(200,200), batch_size=90, class_mode='binary')
                                             

To show how it seperates between floods and landslides

In [ ]:
train_dataset.class_indices

In [ ]:
train_dataset.classes

Defining our model.Creating a convolutional Neural Network with max pooling

In [ ]:
models =tf.keras.models.Sequential([tf.keras.layers.Conv2D(16,(3,3),activation='relu',
                                                          input_shape=(200,200,3)),
                                                          tf.keras.layers.MaxPool2D(2,2),
                                   #
                                   tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
                                                          tf.keras.layers.MaxPool2D(2,2),
                                   #
                                   tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                                                          tf.keras.layers.MaxPool2D(2,2),
                                   ##
                                   tf.keras.layers.Flatten(),
                                   ##
                                   tf.keras.layers.Dense(512,activation='relu'),
                                   ##
                                   tf.keras.layers.Dense(1,activation='sigmoid')
        
                                  ])

In [ ]:
models.compile(loss ='binary_crossentropy',
             optimizer = RMSprop(learning_rate = 0.001),
              metrics = ['accuracy'])

epochs means the number of iterations...so according to this, it iterates 10 times
increasing the epochs increases the accuracy so ,i'll increase from 10 >> 90

*   10 - 60%
*   30 - 70%
*   90 - 90%

In [ ]:
model_fit = models.fit(train_dataset, steps_per_epoch = 3, epochs = 90, validation_data=validation_dataset)                   

ACCURACY OF THIS MODEL

In [ ]:
_, accuracy=models.evaluate(train_dataset)
print('Accuracy:%.2f' % (accuracy*100))

Looping to print all the images in the folder (optional step)

In [ ]:
dir_path ='/content/drive/MyDrive/datasets/image classification/basedata/testing'

for i in os.listdir(dir_path):
    img = image.load_img(dir_path+'//'+i)
    plt.imshow(img)
    plt.show()

checking the data in 1s and 0s again

In [ ]:
validation_dataset.class_indices

Predicting!

In [ ]:
dir_path ='/content/drive/MyDrive/datasets/image classification/basedata/testing/'

for i in os.listdir(dir_path):
    img = image.load_img(dir_path+'//'+ i,target_size=(200,200))
    plt.imshow(img)
    plt.show()
    
    X= image.img_to_array(img)
    X=np.expand_dims(X,axis = 0)
    images = np.vstack([X])
    val = models.predict(images)
    if val == 0:
        print("This is a FLOOD IMAGE")
    elif (val== 1):
        print("This is a LANDSLIDE IMAGE")
    else:
        print("NONE")
